## 引入套件

In [336]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup as bs
import json

## 取得2015/12/31 以前的etf名稱

In [337]:
def get_etf():
    etf_csv=pd.read_csv('Developed Asia Pacific ETF List (108).csv') #讀資料夾底下的csv
    symbol=etf_csv['Symbol']#all etf
    inception=etf_csv['Inception'] # all etf inception time
    count=0
    real_symbol=[] #etf
    in_s=[] #time
    for candiate in inception:
        s=datetime.datetime.strptime(candiate, "%d/%m/%Y")
        if s<=datetime.datetime.strptime("31/12/2015", "%d/%m/%Y"):
           # print(s.strftime("%d/%m/%Y"))
            in_s.append(s.strftime("%d/%m/%Y"))
            real_symbol.append(symbol[count])
        count=count+1
    return real_symbol

## 取得時間區間與每個etf的close值

In [338]:
etfs=get_etf()
etfs.remove('ZJPN')
etfs.remove('PAF')# 移除不合規定的etf
now=datetime.datetime.now()
import time
now = int(time.mktime(now.timetuple()))
start_time=int(time.mktime(datetime.date(2016, 1, 1).timetuple()))

dict={}
date_arr=[]
page_url="https://query1.finance.yahoo.com/v8/finance/chart/"+etfs[0]+"?symbol="+etfs[0]+"&period1="+str(start_time)+"&period2="+str(now)+"&interval=1d&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=HLYrppuGGqQ&corsDomain=finance.yahoo.com&fbclid=IwAR298rNC7s-eA4POkUD5sVlKNEB2YuS4zL9WN23CPyO8CzyUYAF8ezIT9rM"
r = requests.get(page_url)
soup = BeautifulSoup(r.text, 'lxml')
a=soup.p.get_text()
python_dict = json.loads(a)
timestamp=python_dict['chart']['result'][0]['timestamp']
for time in timestamp:
    date_arr.append(datetime.datetime.fromtimestamp(time).strftime("%Y-%m-%d"))
    dict['Date']=date_arr
dict['Date'].pop() #移除最後一個

# 爬etf的close
for etf in etfs:
    page_url="https://query1.finance.yahoo.com/v8/finance/chart/"+etf+"?symbol="+etf+"&period1="+str(start_time)+"&period2="+str(now)+"&interval=1d&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=HLYrppuGGqQ&corsDomain=finance.yahoo.com&fbclid=IwAR298rNC7s-eA4POkUD5sVlKNEB2YuS4zL9WN23CPyO8CzyUYAF8ezIT9rM"
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text, 'lxml')
    a=soup.p.get_text()
    python_dict = json.loads(a)
    dict[etf]=python_dict['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']

## 調整dict長度

In [339]:
for etf in etfs:
    if len(dict[etf])==808:
        dict[etf].pop() #調整符合長度
#     print(etf,len(dict[etf]))

In [340]:
data=pd.DataFrame(dict)
data

,Date,EWJ,EWY,FNDF,VPL,DXJ,EWT,EWH,EPP,FNDC,...,HFXJ,EZJ,ISZE,SCIJ,FHK,HJPX,SCIX,FKO,FAUS,HAUD
0,2015-12-31,46.485207,47.019604,21.969828,52.185001,46.573704,23.509829,17.884832,33.753376,24.937567,...,17.614700,28.050623,20.226845,23.067440,29.359655,24.890738,18.491716,21.622990,24.178898,18.420948
1,2016-01-04,45.756477,45.921497,21.637789,51.337811,45.560020,22.791828,17.496815,33.199459,24.515846,...,17.156073,27.007235,20.226845,23.003695,28.626106,24.139606,18.160885,21.622990,24.178898,18.420948
2,2016-01-05,46.331791,46.120296,21.573227,51.604866,45.773914,22.718189,17.559978,33.023621,24.543962,...,17.429735,27.554377,20.226845,23.003695,28.809494,24.499502,18.160885,21.622990,23.233070,18.317019
3,2016-01-06,45.526352,45.202057,21.195074,50.628757,44.760231,22.202703,17.244156,32.188358,24.112871,...,17.429735,26.469633,20.226845,23.003695,28.294260,23.996218,18.160885,21.622990,22.149658,18.888615
4,2016-01-07,44.835972,44.482613,20.715462,49.625015,43.579151,21.687216,16.910278,31.018988,23.775501,...,16.882406,25.865232,20.226845,23.003695,27.744093,23.312510,18.160885,21.023407,21.564966,18.888615
5,2016-01-08,43.877121,44.236481,20.475660,48.787037,42.742165,21.576757,16.630545,30.605755,23.400639,...,16.504934,25.092234,20.226845,22.001951,26.669958,23.312510,17.281570,21.023407,21.564966,18.888615
6,2016-01-11,44.183964,44.728737,20.521776,49.054085,43.104855,21.668806,16.540310,30.693676,23.410009,...,16.504934,25.127226,20.226845,22.001951,26.591362,23.312510,17.281570,21.023407,21.564966,18.888615
7,2016-01-12,43.953835,44.596207,20.614010,48.934372,42.890957,21.631987,16.549332,30.834352,23.391266,...,16.504934,25.069967,20.226845,22.001951,26.809685,23.312510,17.830051,20.709343,21.435986,18.888615
8,2016-01-13,43.570293,44.473145,20.281971,48.409481,42.602665,21.429474,16.450075,30.350779,23.128866,...,16.504934,24.770947,20.226845,22.001951,26.495304,23.312510,17.830051,21.051960,21.435986,18.888615
9,2016-01-14,44.107254,44.955929,20.577112,49.044876,43.142052,21.834497,16.558357,30.834352,23.222580,...,16.391693,25.069967,20.226845,22.001951,26.416710,22.628805,17.830051,20.890171,21.169436,17.598196
